In [ ]:
%%bash
transeq ~/Downloads/seqdump.txt > protein

In [10]:
from fasta_iterator_class import FASTAReader

ModuleNotFoundError: No module named 'fasta_iterator_class.py'; 'fasta_iterator_class' is not a package

In [12]:
import sys

class FASTAReader(object):

    def __init__(self, file):
        self.last_id = None
        self.file = file
        self.eof = False

    def __iter__(self):
        return self

    def __next__(self):
        if self.eof:
            raise StopIteration
        # check if this is the first sequence from the file
        if self.last_id is None:
            # First line
            line = self.file.readline()
            # Verify that this is a FASTA file
            assert line.startswith(">"), "Not a FASTA file"
            # Get the sequence ID
            seq_id = line[1:].rstrip("\r\n")
        else:
            # Get ID from previous round
            seq_id = self.last_id

        sequence = []
        while True:
            line = self.file.readline()
            # Check if we've reached the end of the file
            if line == "":
                self.eof = True
                break
            # Check if we've reached the next sequence
            elif not line.startswith(">"):
                sequence.append(line.strip())
            # We've reached the next sequence ID
            else:
                self.last_id = line[1:].rstrip("\r\n")
                break
        
        sequence = "".join(sequence)
        return seq_id, sequence

if __name__ == "__main__":
    reader = FASTAReader(sys.stdin)

    for seq_id, sequence in reader:
        print(seq_id, sequence)


AssertionError: Not a FASTA file

In [19]:
def FASTAReader(file):
    line = file.readline()
    assert line.startswith('>'), "Not a FASTA file"
    seq_id = line[1:].rstrip('\r\n')
    sequence = []
    line = file.readline()
    sequences = []
    while line:
        if line.startswith('>'):
            sequences.append((seq_id, ''.join(sequence)))
            seq_id = line[1:].rstrip('\r\n')
            sequence = []
        else:
            sequence.append(line.strip())
        line = file.readline()
    sequences.append((seq_id, ''.join(sequence)))
    return sequences

In [22]:
read_aa_seq = FASTAReader(open("protein_align.fa"))
read_nuc_seq = FASTAReader(open("seqdump_1.txt"))

In [24]:
seq_dict = {}

#make dictionary with original DNA sequence(key) and amino acid sequence(value), then later append dictionary with third query sequence
for seq_id,sequence in read_nuc_seq:
    seq_id = seq_id.split()
    seq_dict[seq_id[0]] = [sequence]
    
for seq_id,sequence in read_aa_seq:
    seq_id = seq_id.split()
    seq_dict[seq_id[0][:-2]].append(sequence)

seq_dict["NC_001563.2"]
    

#need to remove ID from line in sequence
#for sequence id in fasta reader (object) 
#put into a dictionary 
#key is id with list of orignal nuc then amino acid sequence



['AGTAGTTCGCCTGTGTGAGCTGACAAACTTAGTAGTGTTTGTGAGGATTAACAACAATTAACACAGTGCGAGCTGTTTCTTGGCACGAAGATCTCGATGTCTAAGAAACCAGGAGGGCCCGGTAAAAACCGGGCTGTCAATATGCTAAAACGCGGTATGCCCCGCGGATTGTCCTTGATAGGACTAAAGAGGGCTATGCTGAGTCTGATTGACGGGAAGGGCCCAATACGTTTCGTGTTGGCTCTTTTGGCGTTTTTCAGATTCACTGCAATCGCTCCGACTCGTGCGGTGCTGGACAGATGGAGAGGCGTCAACAAACAAACAGCAATGAAGCATCTCTTGAGTTTCAAGAAAGAACTAGGAACTCTGACCAGTGCCATCAACCGCCGGAGCACAAAACAAAAGAAAAGAGGAGGCACAGCGGGCTTTACTATCTTGCTTGGGCTGATCGCCTGTGCTGGAGCTGTGACCCTCTCGAACTTCCAGGGCAAAGTGATGATGACAGTCAATGCAACCGATGTCACTGACGTGATTACCATTCCAACAGCTGCTGGGAAAAACCTGTGCATCGTAAGGGCTATGGACGTAGGATACCTTTGTGAGGATACTATCACTTATGAATGTCCGGTCCTAGCTGCTGGAAATGACCCTGAAGACATTGACTGCTGGTGCACGAAATCATCTGTTTACGTGCGCTATGGAAGATGCACAAAAACTCGGCATTCCCGTCGAAGCAGAAGGTCTCTGACAGTCCAGACACATGGAGAAAGTACACTGGCCAACAAGAAAGGAGCTTGGTTGGACAGCACAAAAGCCACGAGATATCTGGTGAAGACAGAATCATGGATACTGAGAAACCCGGGCTACGCCCTCGTTGCAGCTGTCATTGGATGGATGCTAGGAAGCAACACAATGCAACGCGTCGTGTTTGCCATTCTATTGCTCCTGGTGGCACCAGCATACAGCTTCAACTGTTTAGGAATGAGTAACAGAGACTT

In [ ]:
for key,value in seq_dict.items()
    new_nuc_list: ""
    count = 0
    aa = item[1]
    seq_dict[key].append(new_nuc_list)
    
